In [1]:
import pandas as pd
import numpy as np

from lib.utils import get_AAPL

In [2]:
df = get_AAPL()
df.reset_index(inplace=True)

[*********************100%%**********************]  1 of 1 completed


In [23]:
def hv(series: pd.Series, annualization_factor: int = 252) -> pd.Series:
    """Historical Volatility"""
    return series.diff().rolling(annualization_factor).std() * np.sqrt(annualization_factor)

def rv(series: pd.Series, window: int) -> pd.Series:
    """
    Realized volatility is defined in [Volatility Forecasting with Machine Learning
    and Intraday Commonality](https://arxiv.org/pdf/2202.08962.pdf) as:

    $$RV_{i,t}(h)=\log(\sum_{s=t-h+1}^{t}r^2_{i,s})$$
    """
    assert window > 0, "Window must be greater than 0"
    fuzz = 1e-10
    log_returns = np.log(series).diff() # log returns
    sum_of_squares = log_returns.rolling(window=window).apply(lambda x: np.sum(x**2), raw=True)
    rv = np.log(sum_of_squares + fuzz)
    assert rv.isna().sum() == window, "RV should have NaNs at the beginning" # ? should have one nan from logret and window - 1 from rolling = window
    return rv